# 2.2b Home Energy Usage Model - NS

This notebook simulates the electricity usage of a home's HVAC system using a "baby energy model" that factors in:
- home envelope characteristics (e.g. dimensions, insulation, window area etc)
- HVAC (heating, venting, air conditioning) system characteristics  
- historical weather data

**Out-of-scope:**
- other major energy consumers, like non-HVAC appliances (water heater, fridge, electronics, etc)
- non-electric energy usage (e.g. using natural gas for central heating)
- splitting heating and cooling into separate systems (we assume there is a single HVAC system that does both heating & cooling)

We'll be involving a 3rd party library, called pvlib, to fetch the historical weather data and model solar irradiance.


In [ ]:
# Welcome to your second deep dive lab assignment!

# In this notebook, we're going to simulate the electricity usage of a home's HVAC system
# We're using a "baby energy model" that we're writing from scratch. This is a very simplistic model, so it's only factoring in:
#  - home envelope characteristics (e.g. dimensions, insulation, window area etc)
#  - HVAC (heating, venting, air conditioning) system characteristics
#  - historical weather data

# Out-of-scope
#  - other major energy consumers, like non-HVAC appliances (water heater, fridge, electronics, etc)
#  - non-electric energy usage (e.g. using natural gas for central heating)
#  - splitting heating and cooling into separate systems (we assume there is a single HVAC system that does both heating & cooling)

# We'll be involving a 3rd party library, called pvlib, to fetch the historical weather data and model solar irradiance


In [ ]:
# Coding Level 0-1:
# Run the existing notebook, but update it to use different home attributes.

# In the next cells we're going to:
# 1. Import 3rd party libraries
# 2. Constants (& definitions)
# 3. Inputs: Set input variables (this is where you'll make some changes!)
# 4. Fetch public data: historical weather
# 5. Model: simulate the energy usage of our modeled home over a year of historical weather data


In [ ]:
# In this cell, we're importing packages that we'll use later in the notebook
# You do not need to make changes to this cell.

# Before we can import it, we have to install the pvlib package to the notebook's python kernel
# Hex notebooks have some common 3rd party packages already installed (like pandas), but for less common packages we have to install them first:
# https://pvlib-python.readthedocs.io/
%pip install pvlib --quiet

from dataclasses import dataclass
from enum import Enum
import calendar
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pvlib
import pytz
from pathlib import Path
from typing import NamedTuple


In [ ]:
# In this cell, we define a few permanent constants, as well as a HomeCharacteristics dataclass to help organize our inputs
# You do not need to make changes to this cell

# Also define a few permanent constants
JOULES_PER_KWH = 3.6e+6
JOULES_PER_MEGAJOULE = 1e6
SECONDS_PER_HOUR = 3600
AIR_VOLUMETRIC_HEAT_CAPACITY = 1200 # Energy in joules per cubic meter of air per degree K. (J/m3/K)

# To keep things tidier, we define a HomeCharacteristics dataclass to bunch all the defined and calculated attributes together
@dataclass
class HomeCharacteristics:
    latitude: float
    longitude: float
    heating_setpoint_c: int
    cooling_setpoint_c: int
    hvac_capacity_w: int
    hvac_overall_system_efficiency: int
    conditioned_floor_area_sq_m: int
    ceiling_height_m: int
    wall_insulation_r_value_imperial: int
    ach50: int
    south_facing_window_size_sq_m: int
    window_solar_heat_gain_coefficient: int

    @property
    def building_volume_cu_m(self) -> int:
        return self.conditioned_floor_area_sq_m * self.ceiling_height_m

    @property
    def building_perimeter_m(self) -> float:
        # Assume the building is a 1-story square
        return math.sqrt(self.conditioned_floor_area_sq_m) * 4
    
    @property
    def surface_area_to_area_sq_m(self) -> float:
        # Surface area exposed to air = wall area + roof area (~= floor area, for 1-story building)
        return self.building_perimeter_m * self.ceiling_height_m + self.conditioned_floor_area_sq_m

    @property
    def ach_natural(self) -> float:
        # "Natural" air changes per hour can be roughly estimated from ACH50 with an "LBL_FACTOR"
        # https://building-performance.org/bpa-journal/ach50-achnat/
        LBL_FACTOR = 17
        return self.ach50 / LBL_FACTOR

    @property
    def wall_insulation_r_value_si(self) -> float:
        # The R-values you typically see on products in the US will be in imperial units (ft^2 °F/Btu)
        # But our calculations need SI units (m^2 °K/W)
        return self.wall_insulation_r_value_imperial / 5.67 # SI units: m^2 °K/W

    @property
    def building_heat_capacity(self) -> int:
        # Building heat capacity
        # How much energy (in kJ) do you have to put into the building to change the indoor temperature by 1 degree?
        # Heat capacity unit: Joules per Kelvin degree (kJ/K)
        # A proper treatment of these factors would include multiple thermal mass components,
        # because the walls, air, furniture, foundation, etc. all store heat differently.
        # More info: https://www.greenspec.co.uk/building-design/thermal-mass/
        HEAT_CAPACITY_FUDGE_FACTOR = 1e5
        return self.building_volume_cu_m * HEAT_CAPACITY_FUDGE_FACTOR


In [ ]:
# In this cell, we set various home and HVAC system attributes
# You'll make changes to this cell! Tweak some of the input values to better match your own home's characteristics

home = HomeCharacteristics(
    ## Location
    # Change the two lines below to your home's latitude and longitude. (Find on Google Maps: https://support.google.com/maps/answer/18539)
    latitude = 36.1248871, # Las Vegas, NV
    longitude = -115.3398063, # Las Vegas, NV

    ## HVAC temperature setpoints (i.e. your thermostat settings)
    # Your HVAC system will start heating your home if the indoor temperature is below HEATING_SETPOINT_C (house is too cold)
    # It will start cooling your home if the indoor temperature is above COOLING_SETPOINT_C (house is too warm)
    # Change the two lines below to match your thermostat settings
    heating_setpoint_c=20, # ~65f
    cooling_setpoint_c=22, # ~75f

    ## HVAC system characteristics
    hvac_capacity_w=10000,
    # Different types of HVAC systems have different efficiencies (note: this is a hand-wavy approximation):
    #  - Old boiler with uninsulated pipes = ~0.5
    #  - Electric radiator = ~1
    #  - High-efficiency heat pump = ~4 (how can this be higher than 1?? heat pumpts are magical..) 
    hvac_overall_system_efficiency=1,

    ## Home dimensions
    # Note: these are in SI units. If you're used to Imperial: one square meter is 10.7639 sq. ft
    conditioned_floor_area_sq_m=200, # ~2200 sqft
    ceiling_height_m=3, # 10ft ceilings (pretty tall)

    ## Wall Insulation
    # R value (SI): temperature difference (K) required to create 1 W/m2 of heat flux through a surface. Higher = better insulated
    wall_insulation_r_value_imperial=15, # Imperial units: ft^2 °F/Btu

    ## Air changes per hour at 50 pascals.
    # This is a measure of the "leakiness" of the home: 3 is pretty tight, A "passive house" is < 0.6
    # This number is measured in a "blower door test", which pressurizes the home to 50 pascals
    ach50=10,

    ## Window area
    # We're only modeling South-facing windows, as they have the largest effect from solar irradiance (in the Northern hemisphere)
    # We're assuming the window has an R value matching the walls (so we don't have to model it separately)
    # Change the line below to roughly match the size of your south-facing windows
    south_facing_window_size_sq_m=10, # ~110 sq ft
    # Solar Heat Gain Coefficient (SHGC) is a ratio of how much of the sun's energy makes it through the window (0-1)
    # Different types of windows have different values, e.g. a Double-pane, Low-E, H-Gain window SHGC=0.56
    window_solar_heat_gain_coefficient=0.5,
)


In [ ]:
# In this cell, we're using `pvlib` to fetch historical "solar weather" data for our chosen location for a specific year in the past
# "Solar weather" is how much sun we got at this location

# We're going to simulate our home's electricity usage over a year, using historical weather data for an actual year in the past
# 2022 is the most recent year for which we can get historical solar weather data from NREL (but you could choose an earlier year)
SIMULATION_YEAR = 2022

# Note: You'll need to set your NREL API credentials
# Get them from: https://developer.nrel.gov/signup/
NREL_API_KEY = "YOUR_NREL_API_KEY_HERE"  # Replace with your actual API key
NREL_API_EMAIL = "YOUR_EMAIL_HERE"  # Replace with your actual email

solar_weather_timeseries, solar_weather_metadata = pvlib.iotools.get_psm3(
    latitude=home.latitude,
    longitude=home.longitude,
    names=SIMULATION_YEAR,
    api_key=NREL_API_KEY,
    email=NREL_API_EMAIL,
    map_variables=True,
    leap_day=True,
)

solar_position_timeseries = pvlib.solarposition.get_solarposition(
    time=solar_weather_timeseries.index,
    latitude=home.latitude,
    longitude=home.longitude,
    altitude=100, # Assume close to sea level, this doesn't matter much
    temperature=solar_weather_timeseries["temp_air"],
)

window_irradiance = pvlib.irradiance.get_total_irradiance(
    90, # Window tilt (90 = vertical)
    180, # Window compass orientation (180 = south-facing)
    solar_position_timeseries.apparent_zenith,
    solar_position_timeseries.azimuth,
    solar_weather_timeseries.dni,
    solar_weather_timeseries.ghi,
    solar_weather_timeseries.dhi,
)
solar_weather_timeseries


In [ ]:
# In this cell, we put it all together and simulate the electricity usage of our HVAC system, given a year of historical weather
# You do not need to make changes to this cell

# We're modeling the effect of three external sources of energy that can affect the temperature of the home: 
#  1. Conductive heat gain or loss through contact with the walls and roof (we ignore the floor), given outdoor temperature
#  2. Air change heat gain or loss through air changes between air in the house and outside, given outdoor temperature
#  3. Radiant heat gain from sun coming in south-facing windows

# We then model our HVAC system as heating/cooling/off depending on whether the temperature is above or below desired setpoints

def calculate_next_timestep(
    timestamp,
    indoor_temperature_c,
    outdoor_temperature_c,
    irradiance,
    home: HomeCharacteristics,
    dt=pd.Timedelta(minutes=10) # Defaulting to a timestep of 10 minute increments
):
    '''
    This function calculates the ΔT (the change in indoor temperature) during a single timestep given:
      1. Previous indoor temperature
      2. Current outdoor temperature (from historical weather data)
      3. Current solar irradiance through south-facing windows (from historical weather data)
      4. Home and HVAC characteristics
    '''

    temperature_difference_c = outdoor_temperature_c - indoor_temperature_c

    # Calculate energy in to building

    # 1. Energy conducted through walls & roof (in Joules, J)
    # Conduction
    # Q = U.A.dT, where U = 1/R
    # Convection:
    # Q = m_dot . Cp * dT <=> Q = V_dot * Cv * dT (Cv = Rho * Cp)

    power_in_through_surface_w = (
        temperature_difference_c * home.surface_area_to_area_sq_m / home.wall_insulation_r_value_si
    )
    energy_from_conduction_j = power_in_through_surface_w * dt.seconds

    # 2. Energy exchanged through air changes with the outside air (in Joules, J)
    air_change_volume = (
        dt.seconds * home.building_volume_cu_m * home.ach_natural / SECONDS_PER_HOUR
    )
    energy_from_air_change_j = (
        temperature_difference_c * air_change_volume * AIR_VOLUMETRIC_HEAT_CAPACITY
    )

    # 3. Energy radiating from the sun in through south-facing windows (in Joules, J)
    energy_from_sun_j = (
        home.south_facing_window_size_sq_m
        * home.window_solar_heat_gain_coefficient
        * irradiance
        * dt.seconds
    )

    # 4. Energy added or removed by the HVAC system (in Joules, J)
    # HVAC systems are either "on" or "off", so the energy they add or remove at any one time equals their total capacity
    if indoor_temperature_c < home.heating_setpoint_c:
        hvac_mode = "heating"
        energy_from_hvac_j = home.hvac_capacity_w * dt.seconds
    elif indoor_temperature_c > home.cooling_setpoint_c:
        hvac_mode = "cooling"
        energy_from_hvac_j = -home.hvac_capacity_w * dt.seconds
    else:
        hvac_mode = "off"
        energy_from_hvac_j = 0

    total_energy_in_j = (
        energy_from_conduction_j
        + energy_from_air_change_j
        + energy_from_sun_j
        + energy_from_hvac_j
    )

    # ΔT is the change in indoor temperature during this timestep resulting from the total energy input
    delta_t = total_energy_in_j / home.building_heat_capacity

    return pd.Series(
        {
            "timestamp": timestamp,
            "temperature_difference_c": temperature_difference_c,
            "Conductive energy (J)": energy_from_conduction_j,
            "Air change energy (J)": energy_from_air_change_j,
            "Radiant energy (J)": energy_from_sun_j,
            "HVAC energy (J)": energy_from_hvac_j,
            "hvac_mode": hvac_mode,
            "Net energy xfer": total_energy_in_j,
            "ΔT": delta_t,
            "Outdoor Temperature (C)": outdoor_temperature_c,
            "Indoor Temperature (C)": indoor_temperature_c + delta_t,
            # Actual energy consumption from the HVAC system:
            "HVAC energy use (kWh)": abs(energy_from_hvac_j) / (JOULES_PER_KWH * home.hvac_overall_system_efficiency)
        }
    )

# Since we're starting in January, let's assume our starting temperature is the heating setpoint
previous_indoor_temperature_c = home.heating_setpoint_c

timesteps = []
for timestamp in solar_weather_timeseries.index:
    new_timestep = calculate_next_timestep(
        timestamp=timestamp,
        indoor_temperature_c=previous_indoor_temperature_c,
        outdoor_temperature_c=solar_weather_timeseries.loc[timestamp].temp_air,
        irradiance=window_irradiance.loc[timestamp].poa_direct,
        home=home,
    )
    timesteps.append(new_timestep)
    previous_indoor_temperature_c = new_timestep["Indoor Temperature (C)"]


baby_energy_model = pd.DataFrame(timesteps)
baby_energy_model


In [ ]:
# For each month, let's look at the overall energy balance:
# Where is the thermal energy in the house coming from, and where is it going to?
energy_transfer_columns = [col for col in baby_energy_model.columns if col.endswith("(J)")]
get_month=lambda idx: baby_energy_model.loc[idx]['timestamp'].month
monthly_energy_balance_mj = baby_energy_model.groupby(by=get_month)[energy_transfer_columns].sum() / JOULES_PER_MEGAJOULE

monthly_energy_balance_mj['month'] = monthly_energy_balance_mj.index.map(lambda month_idx: f'{month_idx:0=2} - {calendar.month_name[month_idx]}')

monthly_energy_balance_tidy = monthly_energy_balance_mj.melt(id_vars='month')


## Monthly energy balance

**Note:** The original Hex notebook had an interactive chart here. Here's a matplotlib visualization:


In [ ]:
# Create a stacked bar chart for monthly energy balance
fig, ax = plt.subplots(figsize=(14, 8))

if not monthly_energy_balance_tidy.empty:
    # Pivot the data for plotting
    pivot_data = monthly_energy_balance_tidy.pivot(index='month', columns='variable', values='value')
    
    # Define colors for each energy type
    colors = {
        'Conductive energy (J)': '#E45756',
        'Air change energy (J)': '#4C78A8', 
        'Radiant energy (J)': '#EECA3B',
        'HVAC energy (J)': '#B279A2'
    }
    
    # Create stacked bar chart
    bottom = np.zeros(len(pivot_data.index))
    for col in pivot_data.columns:
        if col in colors:
            ax.bar(pivot_data.index, pivot_data[col], bottom=bottom, 
                  label=col, color=colors[col], alpha=0.8)
            bottom += pivot_data[col]
    
    ax.set_xlabel('Month')
    ax.set_ylabel('Heat energy added to building (MJ)')
    ax.set_title('Monthly Energy Balance')
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax.grid(True, alpha=0.3)
    
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("No data available to plot. Please check your API credentials and data connection.")


## Monthly energy consumption (from HVAC system)

**Note:** The original Hex notebook had an interactive chart here. Here's a matplotlib visualization:


In [ ]:
# Create a monthly energy consumption chart
fig, ax = plt.subplots(figsize=(12, 6))

if not baby_energy_model.empty:
    # Group by month and sum the HVAC energy use
    monthly_hvac = baby_energy_model.groupby(baby_energy_model['timestamp'].dt.to_period('M'))['HVAC energy use (kWh)'].sum()
    
    # Create bar chart
    ax.bar(range(len(monthly_hvac)), monthly_hvac.values, color='#4C78A8', alpha=0.7)
    ax.set_xlabel('Month')
    ax.set_ylabel('HVAC Energy Use (kWh)')
    ax.set_title('Monthly HVAC Energy Consumption')
    ax.grid(True, alpha=0.3)
    
    # Set x-axis labels
    ax.set_xticks(range(len(monthly_hvac)))
    ax.set_xticklabels([str(period) for period in monthly_hvac.index], rotation=45)
    
    plt.tight_layout()
    plt.show()
else:
    print("No data available to plot. Please check your API credentials and data connection.")


## 1 week in summer - indoor vs. outdoor temperature

**Note:** The original Hex notebook had an interactive chart here. Here's a matplotlib visualization:


In [ ]:
# Create a detailed plot for one week in summer
fig, ax1 = plt.subplots(figsize=(14, 8))

if not baby_energy_model.empty:
    # Select one week in summer (August 1-7, 2022 as in original)
    start_date = '2022-08-01'
    end_date = '2022-08-07'
    week_data = baby_energy_model.loc[start_date:end_date]
    
    if not week_data.empty:
        # Plot outdoor temperature
        ax1.plot(week_data.index, week_data['Outdoor Temperature (C)'], 
                color='blue', linewidth=2, label='Outdoor Temperature')
        
        # Plot indoor temperature
        ax1.plot(week_data.index, week_data['Indoor Temperature (C)'], 
                color='orange', linewidth=2, label='Indoor Temperature')
        
        # Create secondary y-axis for HVAC mode
        ax2 = ax1.twinx()
        
        # Map HVAC modes to numeric values for plotting
        hvac_mode_map = {'off': 0, 'heating': 1, 'cooling': -1}
        hvac_numeric = week_data['hvac_mode'].map(hvac_mode_map)
        
        # Plot HVAC mode as scatter
        scatter = ax2.scatter(week_data.index, hvac_numeric, 
                            c=week_data['hvac_mode'].map({'off': 'green', 'heating': 'red', 'cooling': 'blue'}),
                            alpha=0.7, s=50, label='HVAC Mode')
        
        ax1.set_xlabel('Date')
        ax1.set_ylabel('Temperature (°C)')
        ax2.set_ylabel('HVAC Mode')
        ax1.set_title(f'1 Week in Summer - Indoor vs Outdoor Temperature ({start_date} to {end_date})')
        
        # Format x-axis
        ax1.xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%m-%d %H:%M'))
        ax1.xaxis.set_major_locator(plt.matplotlib.dates.HourLocator(interval=12))
        plt.xticks(rotation=45)
        
        # Add legends
        lines1, labels1 = ax1.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
        
        ax1.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
    else:
        print(f"No data available for the selected week ({start_date} to {end_date})")
else:
    print("No data available to plot. Please check your API credentials and data connection.")


## 1 week in winter - indoor vs. outdoor temperature

**Note:** The original Hex notebook had an interactive chart here. Here's a matplotlib visualization:


In [ ]:
# Create a detailed plot for one week in winter
fig, ax1 = plt.subplots(figsize=(14, 8))

if not baby_energy_model.empty:
    # Select one week in winter (February 1-7, 2022 as in original)
    start_date = '2022-02-01'
    end_date = '2022-02-07'
    week_data = baby_energy_model.loc[start_date:end_date]
    
    if not week_data.empty:
        # Plot outdoor temperature
        ax1.plot(week_data.index, week_data['Outdoor Temperature (C)'], 
                color='blue', linewidth=2, label='Outdoor Temperature')
        
        # Plot indoor temperature
        ax1.plot(week_data.index, week_data['Indoor Temperature (C)'], 
                color='orange', linewidth=2, label='Indoor Temperature')
        
        # Create secondary y-axis for HVAC mode
        ax2 = ax1.twinx()
        
        # Map HVAC modes to numeric values for plotting
        hvac_mode_map = {'off': 0, 'heating': 1, 'cooling': -1}
        hvac_numeric = week_data['hvac_mode'].map(hvac_mode_map)
        
        # Plot HVAC mode as scatter
        scatter = ax2.scatter(week_data.index, hvac_numeric, 
                            c=week_data['hvac_mode'].map({'off': 'green', 'heating': 'red', 'cooling': 'blue'}),
                            alpha=0.7, s=50, label='HVAC Mode')
        
        ax1.set_xlabel('Date')
        ax1.set_ylabel('Temperature (°C)')
        ax2.set_ylabel('HVAC Mode')
        ax1.set_title(f'1 Week in Winter - Indoor vs Outdoor Temperature ({start_date} to {end_date})')
        
        # Format x-axis
        ax1.xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%m-%d %H:%M'))
        ax1.xaxis.set_major_locator(plt.matplotlib.dates.HourLocator(interval=12))
        plt.xticks(rotation=45)
        
        # Add legends
        lines1, labels1 = ax1.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
        
        ax1.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
    else:
        print(f"No data available for the selected week ({start_date} to {end_date})")
else:
    print("No data available to plot. Please check your API credentials and data connection.")


In [ ]:
# If you're at coding level 0 or 1: Congrats, you made it through the code!

# Now, go back play around with the data and look for interesting findings. Ideas:
#  - There's a lot of interesting things going on in the "Monthly energy balance" chart. Describe what's going on, and why!
#  - For many locations, the "Monthly energy consumption" chart is bimodal (i.e. it has two peaks at different times of the year). Why?
#  - Tweak various inputs to see how they affect the output. How significant is the impact of:
#    - Higher or lower R-value
#    - More or less air leakiness (i.e. higher or lower ach50 values)
#    - Larger or smaller south-facing windows
#    - Different heating & cooling setpoints

# In your Assignment submission on the Terra.do app:
#  1. Link to your copy of this notebook
#  2. Write up a few sentences summarizing your interesting finding (you can share this in slack as well!)


In [ ]:
# Coding Level 2-3 - Carry on!

# If you have strong programming experience, use this as a starting point and modify or expand this notebook. Some ideas:
#  - The biggest weakness in this model is how thermal mass is mostly ignored. Integrate a more realistic thermal mass component.
#  - We didn't get hands-on with OpenStudio/EnergyPlus/Resstock because of the difficulty of introducing all the dependencies (they're in Java+Ruby).
#       However, NREL has a bleeding-edge, all-python residential energy modeling project: https://github.com/NREL/OCHRE
#       Install their project (%pip install ochre-nrel), fire it up, and compare it to the toy model.
#  - Calculate your estimated electricy costs (due to HVAC electricity usage)
#    (US average electricity price is around $USD 0.23 per kilowatt-hour, but this can vary widely by location)
#  - Fetch TMY data (Typical Meterological Year, averaged across several years) instead of a data from a specific year
#  - Iterate over one or more attributes (programmatically, with charts to compare the outputs)
#  - Look up your actual electricity usage & compare to these simulated results (you might expect HVAC to be around 30-50% of total usage)
#  - Improve this model - account for more factors!
#  - Combine this notebook with the rooftop solar notebook to model what capacity PV system you would need to operate off-grid, 
#    i.e. meet your energy usage for each hour of the year, not just in total (accounting for high usage but low generation in winter)
